### [그룹화 처리]
- 주제 : 타이타닉에서 선실에 등급에 따른 생존 비율을 분석


In [189]:
# [1] 모듈로딩
import pandas as pd
import df_util as util

In [190]:
# [2] 데이터 준비
file_name = 'titanic.csv'

In [191]:
# [3] csv ==> DataFrame으로 저장
dataDF=pd.read_csv(file_name)

In [192]:
# [4] 데이터 확인
util.checkDataFrame(dataDF, 'dataDF')


[dataDF]
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 15 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   survived     891 non-null    int64  
 1   pclass       891 non-null    int64  
 2   sex          891 non-null    object 
 3   age          714 non-null    float64
 4   sibsp        891 non-null    int64  
 5   parch        891 non-null    int64  
 6   fare         891 non-null    float64
 7   embarked     889 non-null    object 
 8   class        891 non-null    object 
 9   who          891 non-null    object 
 10  adult_male   891 non-null    bool   
 11  deck         203 non-null    object 
 12  embark_town  889 non-null    object 
 13  alive        891 non-null    object 
 14  alone        891 non-null    bool   
dtypes: bool(2), float64(2), int64(4), object(7)
memory usage: 92.4+ KB
[Index] : RangeIndex(start=0, stop=891, step=1)
[Columns] : Index(['survived', 'pclass', 'sex', 'ag

In [193]:
# [5] 분석 주제 ==> 선실에 등급에 따른 생존 비율을 분석
# - 분석 컬럼 즉, 선실과 생존 컬럼
# - loc[행,열]
pc_aliveDF=dataDF.loc[:,['sex','age']]            # OR dataDF[['sex','age']]

In [194]:
pc_aliveDF

,sex,age
0,male,22.0
1,female,38.0
2,female,26.0
3,female,35.0
4,male,35.0
...,...,...
886,male,27.0
887,female,19.0
888,female,NaN
889,male,26.0


In [195]:
# [5-1] 데이터 전처리 : 결측치, 중복값
# --- 결측치 : isna(), isnull()
pc_aliveDF.isna().sum()         # False => 정상데이터

sex      0
age    177
dtype: int64

In [196]:
groupObject=pc_aliveDF.groupby('sex') 
groupObject


In [197]:
# -- Group 객체의 속성
# -- Groups 속성 : 그룹화된 그룹에 속하는 인덱스 정보 저장 Dict 형태
print( groupObject.groups.keys() )
print( groupObject.groups.values())

dict_keys(['female', 'male'])
dict_values([Index([  1,   2,   3,   8,   9,  10,  11,  14,  15,  18,
       ...
       866, 871, 874, 875, 879, 880, 882, 885, 887, 888],
      dtype='int64', length=314), Index([  0,   4,   5,   6,   7,  12,  13,  16,  17,  20,
       ...
       873, 876, 877, 878, 881, 883, 884, 886, 889, 890],
      dtype='int64', length=577)])


In [198]:
print(groupObject.indices)
print( groupObject.indices.keys() )

{'female': array([  1,   2,   3,   8,   9,  10,  11,  14,  15,  18,  19,  22,  24,
        25,  28,  31,  32,  38,  39,  40,  41,  43,  44,  47,  49,  52,
        53,  56,  58,  61,  66,  68,  71,  79,  82,  84,  85,  88,  98,
       100, 106, 109, 111, 113, 114, 119, 123, 128, 132, 133, 136, 140,
       141, 142, 147, 151, 156, 161, 166, 167, 172, 177, 180, 184, 186,
       190, 192, 194, 195, 198, 199, 205, 208, 211, 215, 216, 218, 229,
       230, 233, 235, 237, 240, 241, 246, 247, 251, 254, 255, 256, 257,
       258, 259, 264, 268, 269, 272, 274, 275, 276, 279, 289, 290, 291,
       293, 297, 299, 300, 303, 306, 307, 309, 310, 311, 312, 315, 316,
       318, 319, 322, 323, 325, 327, 328, 329, 330, 334, 337, 341, 345,
       346, 347, 356, 357, 358, 359, 362, 366, 367, 368, 369, 374, 375,
       376, 380, 381, 383, 387, 389, 393, 394, 396, 399, 402, 404, 409,
       412, 415, 416, 417, 419, 423, 426, 427, 431, 432, 435, 436, 437,
       440, 443, 446, 448, 457, 458, 469, 472, 473, 4

In [199]:
# -- get_group() : 그룹화된 그룹들 중에서 특정 그룹 데이터 읽기 DataFrame 반환
for key in groupObject.groups.keys():
    print(f'\n[{key}]')
    print(groupObject.get_group(key).ndim,groupObject.get_group(key).shape)
    print(groupObject.get_group(key).min(),groupObject.get_group(key).max())


[female]
2 (314, 2)
sex    female
age      0.75
dtype: object sex    female
age      63.0
dtype: object

[male]
2 (577, 2)
sex    male
age    0.42
dtype: object sex    male
age    80.0
dtype: object


In [200]:
# resultDF=groupObject.agg(['min','max','mean','median','sum'])
# resultDF

In [201]:
dataDF.survived.count() - dataDF.survived.sum()

549

In [202]:
~dataDF.survived.astype('bool')

0       True
1      False
2      False
3      False
4       True
       ...  
886     True
887    False
888     True
889    False
890     True
Name: survived, Length: 891, dtype: bool

- 사용자 정의 함수 지정하기

In [203]:
def myfunc(obj):
    print(type(obj),obj.head(3))
    return obj.max()

In [204]:
myfunc(dataDF['age'])

<class 'pandas.core.series.Series'> 0    22.0
1    38.0
2    26.0
Name: age, dtype: float64


80.0

In [205]:
groupObject.agg(['count',myfunc])

<class 'pandas.core.series.Series'> 1    38.0
2    26.0
3    35.0
Name: age, dtype: float64
<class 'pandas.core.series.Series'> 0    22.0
4    35.0
5     NaN
Name: age, dtype: float64


age       
       count myfunc
sex                
female   261   63.0
male     453   80.0

In [206]:
groupObject.agg(['count',myfunc,lambda x:x.mean()])

<class 'pandas.core.series.Series'> 1    38.0
2    26.0
3    35.0
Name: age, dtype: float64
<class 'pandas.core.series.Series'> 0    22.0
4    35.0
5     NaN
Name: age, dtype: float64


age                  
       count myfunc <lambda_0>
sex                           
female   261   63.0  27.915709
male     453   80.0  30.726645